In [5]:
import pandas as pd 
from requests_html import HTMLSession
from lxml import html 
from datetime import date,datetime

In [14]:
today = date.today()
today_sub = '2022-04-08'
today_sub_dt = datetime.strptime(
    today_sub, '%Y-%m-%d'
).date()

True


In [3]:
def fetchSession(url):
    session = HTMLSession()
    r = session.get(url)
    return r

def getTrades(r):
    table = r.html.find('table')[0]
    rows = table.find('tr')
    return rows[1:]

def arr_to_dict(lst):
    it = iter(lst)
    res_dict = dict(zip(it,it))
    return res_dict 

def value_to_ints(value):
    bad_chars = [
        ',','$','-'
    ]
    for c in bad_chars:
        value = value.replace(c,'')
    low, high = [
        int(x) for x in (value.split('  ', 1))
    ]
    return [low,high]

In [26]:
r = fetchSession('https://sec.report/Senate-Stock-Disclosures')
trades = getTrades(r)
n = len(trades)
all_trades = []
l1_head = [
    'trade date', 'file date', 'trade', 'senator'
]
l2_head = [
    'trade type', 'value'
]

In [30]:
info = []
l1_elements = trades[0].find('td')
l2_elements = trades[1].find('td')[:-1]
file_date, trade_date = l1_elements[0].text.split('\n')
trade = l1_elements[1].text
senator = l1_elements[2].text
l1_cleaned = [
    trade_date,file_date,trade,senator
]
for h,e in zip(l1_head, l1_cleaned):
    info.append(h)
    info.append(e)
for h,e in zip(l2_head, l2_elements):
    info.append(h)
    info.append(e.text)
info[9] = info[9].split('\n',1)[0]
info[11] = value_to_ints(info[11])
info = arr_to_dict(info)
print(info)

{'trade date': '2022-03-29', 'file date': '2022-04-08', 'trade': 'Nov 22 CBT Soybeans', 'senator': 'Thomas H Tuberville [Tuberville, Tommy]', 'trade type': 'Purchase', 'value': [1001, 15000]}


In [ ]:
for i in range(0,n,2):
    trade = []
    l1_elements = trades[i].find('td')
    l2_elements = trades[i+1].find('td')[:-1]
    for h,e in zip(l1_head, l1_elements):
        trade.append(h)
        trade.append(e.text)
    for h,e in zip(l2_head, l2_elements):
        trade.append(h)
        trade.append(e.text)